In [13]:
import os
import json
import clevercsv
import urllib

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

colors = ["teal","xkcd:tangerine"]
mpl.rc('font',family='Rasa')
BAR_WIDTH = 0.3
FONT_SIZE = 20
TITLE_SIZE = 26

In [14]:
def get_df_from_folder(folder, normalize=True, subset = None):
    rows_list = []
        
    for file in os.listdir(folder+"annotations/")[:subset]:
        if file.endswith(".json"):
            with open(folder+"annotations/"+file) as ann:
                data = json.load(ann)

            with open(folder+"csv/"+data["file_name"], "rb") as csvfile:
                data["content"] = csvfile.read()
            dialect_folder = f"./dialects/"

            try:
                dialect_file = dialect_folder + data["file_name"]+"_dialect.json"
                if not os.path.exists(dialect_file):
                    dialect_file = dialect_file.replace(" ", "%20")

                with open(dialect_file, "r") as jf:
                    dialect = json.load(jf)["dialect"]
            except Exception as e:
                print("Filename", dialect_file,e)
                if not (data["table_not_comma_delimiter"] or data["table_not_double_quote"] or data["table_not_escape_quote"]):
                    dialect["delimiter"] =","
                    dialect["quotechar"] = '"'
                    dialect["escapechar"] = '"'
                else:
                    print("Resorting to clevercsv")
                    with open(folder+"csv/"+data["file_name"], "r") as csvfile:
                        d = clevercsv.Sniffer().sniff(csvfile.read())
                    dialect = {"delimiter": d.delimiter, "quotechar": d.quotechar,
                    "escapechar": d.escapechar}

            data["delimiter"] = dialect["delimiter"]
            data["quotechar"] = dialect["quotechar"]
            data["escapechar"] = dialect["escapechar"]

            rows_list += [data]

    df = pd.DataFrame(rows_list)
    df["encoding_nonascii"] = (~(df["encoding"] == "ascii")).astype(int)
    df["dimension_nonstd"] = (~(df["dimension"].between(1024, 128*1024*1024))).astype(int)             
    df["table_columns"] = (df["table_columns_less_than_2"] | df["table_columns_more_256"]).astype(int)
    df["table_header"] = (df["table_no_header"] | df["table_multirow_header"]).astype(int)
    df["table_lines"] = (df["table_lines_less_2"] | df["table_lines_more_65k"]).astype(int)
    df["table_notes"] = (df["table_preamble_rows"] | df["table_footnote_rows"]).astype(int)
    df["column_header"] = (df["column_header_unique"] | df["column_header_non_alnum"] | df["column_header_empty"] | df["column_header_long"]).astype(int)
    df["column_boundary"] = (df["column_string_boundary"] | df["column_int_boundary"] | df["column_date_boundary"]).astype(int)

    df["table_dialect"] = (df["table_not_crlf_delimiter"]  |
                             df["table_not_comma_delimiter"] |      
                             df["table_not_double_quote"]    |
                             df["table_not_escape_quote"])
    df["table_structure"] = (df["table_header"] | df["table_lines"] | df["table_columns"])
    
    df["row_inconsistent_dialect"] =(df["row_inconsistent_record_delimiter"] |
        df["row_inconsistent_field_delimiter"] |
        df["row_inconsistent_quotation"] |
        df["row_inconsistent_escape"])

    df = df.set_index("file_name")
    binary = [x for x in df.columns if x not in ["dimension", "encoding", "file_name", "content"]]
    df["non_std"] = (~df[binary].eq(0).all(axis=1)).astype(int)
    binary = [x for x in df.columns if x not in ["dimension", "encoding", "file_name", "content"]]
    
    binary_subset = df[binary]
    binary_subset = binary_subset.apply(pd.Series.value_counts).fillna(0)
    binary_subset = binary_subset.transpose()
    binary_subset = binary_subset.rename(columns={0: "Standard", 1: "Non-standard"})
    if normalize:
        binary_subset = binary_subset/len(df)
    return df, binary_subset

In [15]:
SUBSET = None
survey_full, survey = get_df_from_folder("./", subset = SUBSET)
datasets = list(zip([survey_full], ["survey"]))

First rule of our grammar is:

```python
file = file_payload CRLF{0,1}
```

We find how many files are empty, have 0, 1 or more than 1 CRLF

In [17]:
no_payload = 0
zero_files = 0
one_files = 0
more_files = 0

total = False

for dataset,name in datasets:
    if not total:
        no_payload = 0
        zero_files = 0
        one_files = 0
        more_files = 0
    for cnt in dataset["content"]:
        if cnt !=b'':
            if cnt.endswith(b"\r\n\r\n") or cnt.endswith(b"\n\n") or cnt.endswith(b"\r\r"):
                more_files+=1
            elif cnt.endswith(b"\r\n") or cnt.endswith(b"\n") or cnt.endswith(b"\r"):
                one_files+=1
            else:
                zero_files+=1
        else:
            no_payload += 1
            continue
            
    if not total:
        print(name)
        print("\tEmpty files",no_payload, "\n\tNo CRLF", zero_files, "\n\tOne CRLF", one_files,"\n\tMore than one CRLF", more_files)
        print()
    
if total:
    print("Total")
    print("\tEmpty files",no_payload, "\n\tNo CRLF", zero_files, "\n\tOne CRLF", one_files,"\n\tMore than one CRLF", more_files)
    print()


survey
	Empty files 15 
	No CRLF 184 
	One CRLF 3508 
	More than one CRLF 5



The second rule states
```python
file_payload = header{0,1} data
```

So we sample how many have 0 headers, how many have one, how many have more than one header


In [40]:
total = False
no_header = 0
more_header = 0
one_header = 0
preamble = 0

for ds,name in datasets:
    dataset = ds[ds["content"]!=b'']
    if not total:
        no_header = 0
        more_header = 0
        one_header = 0        
        preamble = 0
        multitable = 0
        multirow = 0
        
    no_header += len(dataset[dataset["table_no_header"]==1])
    preamble += len(dataset[(dataset["table_multirow_header"]==0) & (dataset["table_preamble_rows"]==1)])
    multirow += len(dataset[(dataset["table_multirow_header"]==1) & (dataset["table_preamble_rows"] == 0)])
    multitable +=len(dataset[dataset["table_multiple_tables"]==1])
    
    more_header = len(dataset[(dataset["table_multirow_header"]==1) | 
                              (dataset["table_multiple_tables"]==1) | 
                              (dataset["table_multirow_header"]==1)])
    
    one_header += len(dataset) - no_header - more_header
    
    if not total:
        print(name)
        print("\tNo header",no_header, "\n\tOne header", one_header)
        print("\tMore than one header", more_header)
        print("\t\tPreamble only", preamble)
        print("\t\tMore headers only", multirow)
        print("\t\tMultitable", multitable)
        print()

survey
	No header 470 
	One header 2751
	More than one header 476
		Preamble only 282
		More headers only 94
		Multitable 232



In [19]:
len(dataset[(dataset["table_not_escape_quote"]==1)])

7

The third rule states that 
```python
data= record (CRLF record){0,*}
```

So we sample how many of the files have no record, a single record, or multiple records.
(No record means that it can have a header or be empty)

In [21]:
for ds,name in datasets:
    dataset = ds[ds["content"]!=b'']
    less_than_2_record = len(dataset[dataset["table_lines_less_2"]==1])
    more_records = len(dataset[dataset["table_lines_less_2"]==0])
    
    no_record = len(dataset[dataset["dimension"]==0][dataset["table_lines_less_2"]==0])
    one_record= 0
    for _,f in dataset[dataset["table_lines_less_2"]==1].iterrows():
        if f["table_no_header"]==1:
            one_record +=1
        else:
            no_record +=1
            
    print(name)
    print("\tNo record",no_record, "\n\tOne record", one_record, "\n\tMore records", more_records)
    print()


survey
	No record 3 
	One record 4 
	More records 3690



<ipython-input-21-dfb788c3d337>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_record = len(dataset[dataset["dimension"]==0][dataset["table_lines_less_2"]==0])
<ipython-input-21-dfb788c3d337>:6: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  no_record = len(dataset[dataset["dimension"]==0][dataset["table_lines_less_2"]==0])


The fourth rule and fifth rule state
```python
header = field (COMMA field){N,N} CRLF
record = field (COMMA field){N,N}
```


N is a parameter that depends on a given file, so the pollution here is having files where header and records have a different N, or where a record has a different number of fields from other records.
We did not find a file where header is not separated with crlf from the rest of the data.


In [42]:
from io import BytesIO 

for ds,name in datasets:
    dataset = ds[ds["content"]!=b'']
    inconsistent_n = len(dataset[dataset["row_inconsistent_n_delimiter"]==1])
    inconsistent_n += len(dataset[(dataset["table_multiple_tables"]) & (dataset["row_inconsistent_n_delimiter"]==0)])
    multitable = len(dataset[dataset["table_multiple_tables"]==1])
    preamble = len(dataset[(dataset["table_multiple_tables"]==0) & ((dataset["table_preamble_rows"]==1) | dataset["table_multirow_header"]==1) & (dataset["row_inconsistent_n_delimiter"]==1)])
    consistent_n = len(dataset) - inconsistent_n
                
    print(name)
    print("\tInconsistent N",inconsistent_n, "\n\tConsistent N", consistent_n)
    print("\t Multitable", multitable, "\n\t Preamble", preamble)
    print()  

survey
	Inconsistent N 1040 
	Consistent N 2657
	 Multitable 232 
	 Preamble 221



The last formatting rule states
```python
escaped = DQUOTE text{0,*} DQUOTE
```

A pollution to this formatting rule implies that there is an odd number of quotes inside a field:\
If there is no DQUOTE{2} in text:
- Either because the first quote or the last quote is missing\
- If both quotes are missing and there is a CR, LF, COMMA, this would be reflected in the previous pollution

If there is a DQUOTE{2} in text:
- First quote missing means you have an invalid file because you should have delimiter+quote
- Second quote missing means you *probably* have a row with an inconsistent number of cells, or a quote never closing
- Both quotes missing falls back to invalid file because you should have delimiter-delimiter or delimiter-quote

Sanity check: file has an even number of quotes (this assumes that the non-escaped payload rule is respected!)

In [43]:
import regex as re
from tqdm import tqdm

SUBSET = None

for dataset,name in datasets:
    non_quoted = 0
    correctly_escaped = 0
    polluted_escape = 0
    non_escaped = 0
    cnt_slash = 0
    potential_escape = []
    for fname,f in tqdm(dataset[:SUBSET].iterrows()):
        try:
            quotechar = f.quotechar[0] #exclude case where it's quote with space
        except IndexError:
            quotechar = '"'
        delimiter = f["delimiter"]

        x = f["content"]
        cnt = 0
        rx = re.compile(bytes(f"^.*{quotechar}.*$",encoding="utf-8"), re.MULTILINE)
        for line in rx.findall(x):
            cnt += line.count(bytes(quotechar,encoding="utf-8"))
            cnt_slash +=line.count(bytes(f'\\{quotechar}', encoding="utf-8"))
        if not cnt:
            non_quoted += 1
            continue

        if cnt_slash:
            print("Found backslash in", fname)

        if cnt%2 != 0: #if there is an odd number they are polluted
            polluted_escape+=1
            print("Polluted escape in:",fname)
            continue

        empty_cell = bytes(f"({delimiter}){quotechar}{quotechar}{delimiter}|^{delimiter}{quotechar}{quotechar}|{delimiter}{quotechar}{quotechar}$", encoding="utf-8")
        rx = re.compile(empty_cell, re.MULTILINE)
        x = rx.sub(rb'\1',x)

        rx = re.compile(bytes(f"^.*{quotechar}{quotechar}.*$",encoding="utf-8"), re.MULTILINE)
        if len(rx.findall(x)):
            # print("Escaped quote in:", fname)
            potential_escape.append(fname)
            correctly_escaped += 1
        else:
            non_escaped +=1

    print("\tPolluted escape",polluted_escape, "\n\tUnpolluted escape", correctly_escaped, "\n\tNon escaped", non_escaped)


636it [00:02, 321.27it/s]

In [27]:
potential_escape[:10]

['160930 - Organogram Data - Home Office-senior.csv',
 '1810IPOpaymentsb.csv',
 '1901IPOpaymentsb.csv',
 '1905IPOpaymentsb.csv',
 '1906IPOpaymentsb.csv',
 '20130402_gpc_spend_jan13.csv',
 '2015 thesaurus file.csv',
 '20150608-foi-cases-received-endmarch-2015.csv',
 '2015_September_GPC_report.csv',
 '2016-17_P12_MHRA_GPC_Transparency_Data_-_March_2017.csv']

The second rule states
```python
file_payload = header_row{0,1} data
```

So we sample how many have 0 headers, how many have one, how many have more than one header


The second rule states
```python
file_payload = header_row{0,1} data
```

So we sample how many have 0 headers, how many have one, how many have more than one header


In [28]:
for dataset,name in datasets:
    crlf = len(dataset[dataset["table_not_crlf_delimiter"]==0])
    only_cr = 0
    only_lf = 0        

    for idx,f in dataset.iterrows():
        if not f["table_not_crlf_delimiter"]:
            continue
        if f["content"].endswith(b"\r"):
            only_cr +=1
        elif f["content"].endswith(b"\n"):
            only_lf +=1
        else:
            if re.findall(b".\r.", f["content"], re.MULTILINE):
                only_cr +=1
            elif re.findall(b".\n.", f["content"], re.MULTILINE):
                only_lf +=1
            else:
                print(f["content"][:1000])

    print(name)
    print("\tCRLF",crlf)
    print("\tCR only", only_cr)
    print("\tLF only", only_lf)
    print()

survey
	CRLF 2014
	CR only 7
	LF only 1691



In [35]:
for dataset,name in datasets:
    comma = len(dataset[(dataset["table_not_comma_delimiter"]==0) & (dataset["row_inconsistent_field_delimiter"]==0)])
    
    print(name)
    print("Comma:", comma)
    print(dataset[(dataset["table_not_comma_delimiter"]==1)].delimiter.value_counts())
    print(dataset[(dataset["row_inconsistent_field_delimiter"]==1)].delimiter.value_counts())
    print()

survey
Comma: 2757
;      833
,       91
,\t     11
\t       5
         3
Name: delimiter, dtype: int64
,    12
Name: delimiter, dtype: int64

